![Powrangling.png](attachment:Powrangling.png)

<a id='section_id0'></a>
# BMIS-2542: Data Programming Essentials *with Python*
##### Katz Graduate School of Business, University of Pittsburgh


## Session 3: Merging Datasets

<font color="#740202">**Session-3 Topics:**</font>

|No.|Topic||No.|Topic||No.|Topic|
|:------|:------||:------|:------||:------|:------|
|1.|[Combining DataFrames](#section_id1)||2.|[Matching Data](#section_id1a)||3.|[Handling Dates](#section_id2)|
|4.|[Missing Values Information](#section_id3)  ||5.|[Categorizing Information](#section_id4)  ||6.|[Rank and Rank Percentile](#section_id5)  |
|7.|[Tabulating and Querying](#section_id6)  ||8.|[Additional Resources](#section_id7)  ||||||

***
<font color="#740202">**List of Session-3 Exercises:**</font>

(1) [Combining data](#section_id-ex1), (2) [Date stuff](#section_id-ex2), (3) [Missing values](#section_id-ex3), (4) [Categorization](#section_id-ex4), (5) [Ranking](#section_id-ex5), and (6) [Pivot/Crosstabs](#section_id-ex6)
***

Quick cheat sheets:
(1) The [official Pandas cheat sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf), (2) a one-page [comunity-created cheat sheet](https://datacamp-community-prod.s3.amazonaws.com/dbed353d-2757-4617-8206-8767ab379ab3), and (3) a list of [Pandas commands in one page](https://s3.amazonaws.com/dq-blog-files/pandas-cheat-sheet.pdf)
***

In [1]:
# load required modules
import pandas as pd
import numpy as np
import datetime

<a id='section_id1'></a>
### <span style="background-color:#B0E0E6">Combining DataFrames</span>
Data may be spread across a number of files or databases or be arranged in a form that is not easy to analyze. This section focuses on tools to combine and join data.<br>
Data contained in pandas objects can be combined in a number of ways:
 - **pandas.merge**: connects rows in DataFrames based on one or more keys (similar to database joins)
 - **pandas.concat**: concatenates or "stacks" together objects along a specified axis

#### Database-Style DataFrame Joins
Merge or join operations combine datasets by linking rows using one or more keys.

![Joins.png](attachment:Joins.png)
[<center>Source</center>](https://www.w3schools.com/sql/sql_join.asp)

In [2]:
dfCustomers = pd.DataFrame({'customer_id':[1,2,3], 'name':['A', 'B', 'C']})
dfCustomers

,customer_id,name
0,1,A
1,2,B
2,3,C


In [3]:
dfOrders = pd.DataFrame({'order_id':[1,2,3,4], 'customer_id':[2,2,1,5], 'value':[1000,2000,3000,4000]})
dfOrders

,order_id,customer_id,value
0,1,2,1000
1,2,2,2000
2,3,1,3000
3,4,5,4000


By default, `merge` does an inner join. The keys in the result are the intersection or the common set found on both tables. Other possible options are `left`, `right`, and `outer` joins.

In [4]:
dfMerged = pd.merge(dfOrders, dfCustomers, on='customer_id')
dfMerged

,order_id,customer_id,value,name
0,1,2,1000,B
1,2,2,2000,B
2,3,1,3000,A


The following also works when there is only one common column name between the dataframes.

In [5]:
dfMerged_new = pd.merge(dfOrders, dfCustomers)
dfMerged_new

,order_id,customer_id,value,name
0,1,2,1000,B
1,2,2,2000,B
2,3,1,3000,A


In [6]:
dfMergedLeft = pd.merge(dfOrders, dfCustomers, on='customer_id', how='left')
dfMergedLeft

,order_id,customer_id,value,name
0,1,2,1000,B
1,2,2,2000,B
2,3,1,3000,A
3,4,5,4000,NaN


In [7]:
dfMergedRight = pd.merge(dfOrders, dfCustomers, on='customer_id', how='right')
dfMergedRight

,order_id,customer_id,value,name
0,3.0,1,3000.0,A
1,1.0,2,1000.0,B
2,2.0,2,2000.0,B
3,NaN,3,NaN,C


In [8]:
dfMergedOuter = pd.merge(dfOrders, dfCustomers, on='customer_id', how='outer')
dfMergedOuter

,order_id,customer_id,value,name
0,1.0,2,1000.0,B
1,2.0,2,2000.0,B
2,3.0,1,3000.0,A
3,4.0,5,4000.0,NaN
4,NaN,3,NaN,C


In [9]:
# Suppose the Customer Name column of the Customers DataFrame is "c_id" instead of "customer_id"
dfCustomers2 = pd.DataFrame({'c_id':[1,2,3], 'name':['A', 'B', 'C']})
dfCustomers2

,c_id,name
0,1,A
1,2,B
2,3,C


In [10]:
# If the merging column names are different, they can be specified separately using "left_on" and "right_on"
# (the other option is that we change the name of one of the table and make sure merging columns have same names)
dfMergedDiff = pd.merge(dfOrders, dfCustomers2, left_on ='customer_id', right_on = 'c_id')
dfMergedDiff

,order_id,customer_id,value,c_id,name
0,1,2,1000,2,B
1,2,2,2000,2,B
2,3,1,3000,1,A


In [11]:
dfMergedDiffLeft = pd.merge(dfOrders, dfCustomers2, left_on ='customer_id', right_on = 'c_id', how="left")
dfMergedDiffLeft

,order_id,customer_id,value,c_id,name
0,1,2,1000,2.0,B
1,2,2,2000,2.0,B
2,3,1,3000,1.0,A
3,4,5,4000,NaN,NaN


#### Merging with Multiple Keys

In [12]:
leftTable = pd.DataFrame({'key1':['foo', 'foo','bar'], 'key2':['one', 'two', 'one'], 'left_val':[1,2,3]})
rightTable = pd.DataFrame({'key1':['foo', 'foo','bar','bar'], 'key2':['one', 'one', 'one','two'], 'right_val':[4,5,6,7]})

In [13]:
leftTable

,key1,key2,left_val
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [14]:
rightTable

,key1,key2,right_val
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [15]:
pd.merge(leftTable, rightTable, on=['key1', 'key2'])

,key1,key2,left_val,right_val
0,foo,one,1,4
1,foo,one,1,5
2,bar,one,3,6


In [16]:
dfLeft = pd.DataFrame({'key1':['foo', 'foo','bar'], 'key2':['one', 'two', 'one'], 'left_val':[1,2,3]})
dfRight = pd.DataFrame({'key3':['foo', 'foo','bar','bar'], 'key4':['one', 'one', 'one','two'], 'right_val':[4,5,6,7]})

In [17]:
dfLeft

,key1,key2,left_val
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [18]:
dfRight

,key3,key4,right_val
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


Let's merge `dfLeft` and `dfRight` by an inner join on `dfLeft.key1 = dfRight.key3` and `dfLeft.key2 = dfRight.key4`

In [19]:
pd.merge(dfLeft, dfRight, left_on=['key1', 'key2'], right_on=['key3', 'key4'])

,key1,key2,left_val,key3,key4,right_val
0,foo,one,1,foo,one,4
1,foo,one,1,foo,one,5
2,bar,one,3,bar,one,6


### **Concatenating** Along an Axis

In [20]:
# Combine two DataFrames with identical columns
df1 = pd.DataFrame([['a', 1], ['b', 2]], columns=['letter', 'number'])
df2 = pd.DataFrame([['c', 3], ['d', 4]], columns=['letter', 'number'])

In [21]:
df1

,letter,number
0,a,1
1,b,2


In [22]:
df2

,letter,number
0,c,3
1,d,4


In [23]:
pd.concat([df1, df2])

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4


In [24]:
df3 = pd.DataFrame([['c', 3, 'cat'], ['d', 4, 'dog']], columns=['letter', 'number', 'animal'])
df3

,letter,number,animal
0,c,3,cat
1,d,4,dog


In [25]:
# Combine DataFrame objects with overlapping columns and return everything.
# Columns outside the intersection will be filled with NaN values.
pd.concat([df1, df3], sort=False, ignore_index=True) # ignore_index=True: create new index

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
2,c,3,cat
3,d,4,dog


In [26]:
# Combine DataFrame objects with overlapping columns and return only those that are shared
pd.concat([df1, df3], join="inner", ignore_index=True) # ignore_index=True: create new index

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4


In [27]:
pd.concat([df1, df3], join="outer", ignore_index=True) # ignore_index=True: create new index

,letter,number,animal
0,a,1,NaN
1,b,2,NaN
2,c,3,cat
3,d,4,dog


In [28]:
df4 = pd.DataFrame([['bird', 'polly'], ['monkey', 'george']],  columns=['animal', 'name'])
df4

,animal,name
0,bird,polly
1,monkey,george


In [29]:
# Combine DataFrame objects horizontally, by specifying the axis
pd.concat([df1, df4], axis=1) # axis = 1 means row-wise operation

,letter,number,animal,name
0,a,1,bird,polly
1,b,2,monkey,george


In [30]:
pd.concat([df1, df4], axis=0) # axis = 1 means row-wise operation

,letter,number,animal,name
0,a,1.0,NaN,NaN
1,b,2.0,NaN,NaN
0,NaN,NaN,bird,polly
1,NaN,NaN,monkey,george


<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

### **Matching Information**

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We'll use the "grader.csv" dataset for this example.

In [32]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python Course/All Dataset for Python/grader.csv')

In [33]:
df1.head()

,grader,user,username,assignment,rating,age,timestamp,State
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,AK
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,AK
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,AK
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,AL
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,AL


In [34]:
# graderdf = pd.read_csv('grader.csv')

<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

### **Matching Need:**
Include a new column called “region” that is populated according to the U.S. State
* This is a common issue in data wrangling as showcased by [this Stackoverflow discussion](https://stackoverflow.com/questions/45556197/get-us-census-regions-using-states)
* We'll reuse the mapping others have posted, for example, in this [GitHub file](https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv).

In [35]:
df2=pd.read_csv('https://raw.githubusercontent.com/cphalpert/census-regions/master/us%20census%20bureau%20regions%20and%20divisions.csv')

In [36]:
df2.columns

Index(['State', 'State Code', 'Region', 'Division'], dtype='object')

In [37]:
df2.head()

,State,State Code,Region,Division
0,Alaska,AK,West,Pacific
1,Alabama,AL,South,East South Central
2,Arkansas,AR,South,West South Central
3,Arizona,AZ,West,Mountain
4,California,CA,West,Pacific


In [38]:
graderdf=pd.merge(df1, df2, left_on = 'State', right_on = 'State Code', how='left')

In [39]:
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State_x,State_y,State Code,Region,Division
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,AK,Alaska,AK,West,Pacific
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,AK,Alaska,AK,West,Pacific
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,AK,Alaska,AK,West,Pacific
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,AL,Alabama,AL,South,East South Central
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,AL,Alabama,AL,South,East South Central


Note: Since there were two columns with name "State", the merge function has created two new columns "State_x" and "State_y". We now delete one of these columns (State_x) and rename the other column (State_y) as "State"

In [40]:
graderdf.drop('State_x', axis='columns', inplace=True)

In [41]:
graderdf.rename(columns={'State_y': 'State'}, inplace=True)

In [42]:
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central


<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

### **Handling Dates**
To find out how many assignments were graded in September:
* We need to convert the timestamp to a date value
* The timestamp is in the Unix epoch time (or POSIX time),  an indicator of the number of seconds elapsed since 0 hours UTC on 01 January 1970. [See for more details](https://en.wikipedia.org/wiki/Unix_time)

 #### Date Conversions

In [43]:
graderdf['date']=pd.to_datetime(graderdf['timestamp'], unit='s', errors='coerce') # unit = 's' means time stamp is in seconds, errors='coerce': invalid parsing will be set as NaN.

In [44]:
graderdf.tail()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date
13,2.0,2.0,Johnny,2.0,6.0,45.0,1.567906e+09,New York,NY,Northeast,Middle Atlantic,2019-09-08 01:28:18
14,3.0,2.0,Johnny,2.0,4.0,55.0,1.567906e+09,New York,NY,Northeast,Middle Atlantic,2019-09-08 01:29:58
15,1.0,3.0,Johnson,2.0,10.0,35.0,1.567906e+09,West Virginia,WV,South,South Atlantic,2019-09-08 01:31:38
16,2.0,3.0,Johnson,2.0,3.0,45.0,1.567906e+09,West Virginia,WV,South,South Atlantic,2019-09-08 01:33:18
17,3.0,3.0,Johnson,2.0,NaN,NaN,NaN,West Virginia,WV,South,South Atlantic,NaT


In [45]:
graderdf['year']=pd.to_datetime(graderdf['date']).dt.to_period('Y') # 'Y' means get 'year' from the date. Use 'M' to get the 'month'.

In [46]:
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:44:54,2019
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:46,2019
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:12,2019
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:38,2019


In [47]:
graderdf['month'] = pd.DatetimeIndex(graderdf['date']).month
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:44:54,2019,10.0
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019,10.0
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:46,2019,10.0
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:12,2019,10.0
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:38,2019,10.0


In [48]:
graderdf['Month_Name'] = pd.DatetimeIndex(graderdf['date']).month_name()
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month,Month_Name
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:44:54,2019,10.0,October
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019,10.0,October
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:46,2019,10.0,October
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:12,2019,10.0,October
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:38,2019,10.0,October


Check out the documentation for [DatetimeIndex](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DatetimeIndex.html)

#### Counting data
How many assingments were graded in September?

In [49]:
graderdf[graderdf['month']==9].shape[0]

5

In [50]:
len(graderdf[graderdf['month']==9])

5

<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

<a id='section_id3'></a>
### <span style="background-color:#B0E0E6">Compiling Information on Missing Values</span>

To get an idea of missing values, let us create a dataframe with the following information:

| grader| counts |missing count | missing proportion|
|-------|--------|--------------|-------------------|
|--xx---|--xx----|---xx---------|--xx---------------|


#### Missing Data

In [51]:
graderdf.isna().sum() #missing values in each column

grader        1
user          1
username      0
assignment    2
rating        2
age           1
timestamp     1
State         0
State Code    0
Region        0
Division      0
date          1
year          1
month         1
Month_Name    1
dtype: int64

In [52]:
graderdf.isna().sum().sum() #chain functioning -- total missing values in the dataframe

12

In [53]:
graderdf[graderdf.isna().any(axis=1)].shape[0] #number of rows with atleast one missing values in a record (row)

5

In [54]:
graderdf[graderdf.isna().any(axis=1)].index # Indexes of rows with missing values

Int64Index([1, 6, 10, 11, 17], dtype='int64')

In [55]:
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month,Month_Name
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:44:54,2019,10.0,October
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019,10.0,October
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:46,2019,10.0,October
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:12,2019,10.0,October
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:38,2019,10.0,October


In [56]:
graderdf['missing']=0 #create a dummy variable and initialize all the values to 0

In [57]:
graderdf.loc[graderdf.isna().any(axis=1), 'missing']=1 #mark a row if it is missing at least one missing value

In [58]:
graderdf[graderdf.isna().any(axis=1)]

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month,Month_Name,missing
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019,10.0,October,1
6,1.0,3.0,Johnson,NaN,1.0,35.0,1.570467e+09,California,CA,West,Pacific,2019-10-07 16:47:30,2019,10.0,October,1
10,2.0,1.0,John,2.0,NaN,45.0,1.570467e+09,Pennsylvania,PA,Northeast,Middle Atlantic,2019-10-07 16:49:14,2019,10.0,October,1
11,NaN,1.0,John,2.0,8.0,55.0,1.570467e+09,Pennsylvania,PA,Northeast,Middle Atlantic,2019-10-07 16:49:40,2019,10.0,October,1
17,3.0,3.0,Johnson,2.0,NaN,NaN,NaN,West Virginia,WV,South,South Atlantic,NaT,NaT,NaN,NaN,1


In [59]:
graderdf[graderdf['missing']==1].shape[0]

5

In [60]:
gradergroup1=graderdf.groupby(['grader'])['grader'].agg('count').reset_index(name='counts')
gradergroup1

,grader,counts
0,1.0,6
1,2.0,6
2,3.0,5


In [61]:
gradergroup2=graderdf.groupby(['grader'])['missing'].agg('sum').reset_index(name='missingcount')
gradergroup2

,grader,missingcount
0,1.0,1
1,2.0,2
2,3.0,1


In [62]:
gradergroup=pd.merge(gradergroup1, gradergroup2, how='outer')
gradergroup

,grader,counts,missingcount
0,1.0,6,1
1,2.0,6,2
2,3.0,5,1


In [63]:
gradergroup['missingprop']=gradergroup.missingcount/gradergroup.counts*100
gradergroup

,grader,counts,missingcount,missingprop
0,1.0,6,1,16.666667
1,2.0,6,2,33.333333
2,3.0,5,1,20.000000


<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

### **Categorizing Information**

* Insert a score_category depending on the movie rating:
    * <4 --> ok
    * 4-7 --> good
    * greater than 8 --> excellent

In [64]:
graderdf['score_category']=pd.cut(graderdf['rating'], bins=[1,3,7,10], labels=['ok', 'good', 'excellent'])
graderdf.head()

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month,Month_Name,missing,score_category
0,1.0,1.0,John,1.0,4.0,35.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:44:54,2019,10.0,October,0,good
1,2.0,NaN,John,NaN,4.0,45.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:20,2019,10.0,October,1,good
2,3.0,1.0,John,1.0,7.0,55.0,1.570467e+09,Alaska,AK,West,Pacific,2019-10-07 16:45:46,2019,10.0,October,0,good
3,1.0,2.0,Johnny,1.0,7.0,35.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:12,2019,10.0,October,0,good
4,2.0,2.0,Johnny,1.0,3.0,45.0,1.570467e+09,Alabama,AL,South,East South Central,2019-10-07 16:46:38,2019,10.0,October,0,ok


<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

<a id='section_id5'></a>
### <span style="background-color:#B0E0E6">Deriving Rank and Percentile Rank</span>
Display the rank and percentile score of each user

In [65]:
user_avg=graderdf[['user', 'rating']].groupby('user').mean().reset_index()
user_avg

,user,rating
0,1.0,5.75
1,2.0,4.50
2,3.0,4.60


In [66]:
user_avg.sort_values(by=['rating'])

,user,rating
1,2.0,4.50
2,3.0,4.60
0,1.0,5.75


In [67]:
user_avg['rank']=user_avg['rating'].rank(ascending=False)
user_avg

,user,rating,rank
0,1.0,5.75,1.0
1,2.0,4.50,3.0
2,3.0,4.60,2.0


In [68]:
user_avg['pctrank']=user_avg['rating'].rank(pct=True)
user_avg

,user,rating,rank,pctrank
0,1.0,5.75,1.0,1.000000
1,2.0,4.50,3.0,0.333333
2,3.0,4.60,2.0,0.666667


<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>

<a id='section_id6'></a>
### <span style="background-color:#B0E0E6">Tabulating Information & Querying</span>
Aggregate and display the scores for each user per grader in a table. Then, query the above table for a user with name containing “son”!

In [69]:
pd.crosstab(graderdf.username, graderdf.grader, values=graderdf.rating, aggfunc=np.mean)

grader,1.0,2.0,3.0
username,,,
John,4.0,4.0,7.0
Johnny,4.5,4.5,4.5
Johnson,5.5,2.0,8.0


In [71]:
table=pd.pivot_table(graderdf, values='rating', index=['username', 'grader'], aggfunc=np.mean)
table

rating
username grader        
John     1.0        4.0
         2.0        4.0
         3.0        7.0
Johnny   1.0        4.5
         2.0        4.5
         3.0        4.5
Johnson  1.0        5.5
         2.0        2.0
         3.0        8.0

In [72]:
table=pd.pivot_table(graderdf, values='rating', index=['username', 'grader'], aggfunc=np.mean).reset_index()
table

,username,grader,rating
0,John,1.0,4.0
1,John,2.0,4.0
2,John,3.0,7.0
3,Johnny,1.0,4.5
4,Johnny,2.0,4.5
5,Johnny,3.0,4.5
6,Johnson,1.0,5.5
7,Johnson,2.0,2.0
8,Johnson,3.0,8.0


##### Querying the table

In [73]:
table.query('username==["Johnson"]')

,username,grader,rating
6,Johnson,1.0,5.5
7,Johnson,2.0,2.0
8,Johnson,3.0,8.0


In [74]:
graderdf[graderdf.username.str.contains('son')]

,grader,user,username,assignment,rating,age,timestamp,State,State Code,Region,Division,date,year,month,Month_Name,missing,score_category
6,1.0,3.0,Johnson,NaN,1.0,35.0,1.570467e+09,California,CA,West,Pacific,2019-10-07 16:47:30,2019,10.0,October,1,NaN
7,2.0,3.0,Johnson,1.0,1.0,45.0,1.570467e+09,California,CA,West,Pacific,2019-10-07 16:47:56,2019,10.0,October,0,NaN
8,3.0,3.0,Johnson,1.0,8.0,55.0,1.570467e+09,California,CA,West,Pacific,2019-10-07 16:48:22,2019,10.0,October,0,excellent
15,1.0,3.0,Johnson,2.0,10.0,35.0,1.567906e+09,West Virginia,WV,South,South Atlantic,2019-09-08 01:31:38,2019,9.0,September,0,excellent
16,2.0,3.0,Johnson,2.0,3.0,45.0,1.567906e+09,West Virginia,WV,South,South Atlantic,2019-09-08 01:33:18,2019,9.0,September,0,ok
17,3.0,3.0,Johnson,2.0,NaN,NaN,NaN,West Virginia,WV,South,South Atlantic,NaT,NaT,NaN,NaN,1,NaN


In [75]:
table.reset_index()[table.reset_index().username.str.contains('son')]

,index,username,grader,rating
6,6,Johnson,1.0,5.5
7,7,Johnson,2.0,2.0
8,8,Johnson,3.0,8.0


### Creating New Dataset by Filtering

In [76]:
world_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python Course/Large Data Sets/worlddata.csv')
world_data

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06
...,...,...,...,...,...,...
5656453,Zimbabwe,ZWE,Time required to register property (days),IC.PRP.DURS,2015,3.600000e+01
5656454,Zimbabwe,ZWE,Time required to start a business (days),IC.REG.DURS,2015,9.000000e+01
5656455,Zimbabwe,ZWE,Time to prepare and pay taxes (hours),IC.TAX.DURS,2015,2.420000e+02
5656456,Zimbabwe,ZWE,Time to resolve insolvency (years),IC.ISV.DURS,2015,3.300000e+00


In [77]:
# world_data.reset_index(inplace=True) # Adding "index" column

In [78]:
Indicator_Values = world_data.IndicatorName.unique()
Indicator_Values

array(['Adolescent fertility rate (births per 1,000 women ages 15-19)',
       'Age dependency ratio (% of working-age population)',
       'Age dependency ratio, old (% of working-age population)', ...,
       'Fish species, threatened', 'Mammal species, threatened',
       'Plant species (higher), threatened'], dtype=object)

In [102]:
len(Indicator_Values)

1344

In [79]:
for i in Indicator_Values:
  print(i)

Adolescent fertility rate (births per 1,000 women ages 15-19)
Age dependency ratio (% of working-age population)
Age dependency ratio, old (% of working-age population)
Age dependency ratio, young (% of working-age population)
Arms exports (SIPRI trend indicator values)
Arms imports (SIPRI trend indicator values)
Birth rate, crude (per 1,000 people)
CO2 emissions (kt)
CO2 emissions (metric tons per capita)
CO2 emissions from gaseous fuel consumption (% of total)
CO2 emissions from liquid fuel consumption (% of total)
CO2 emissions from liquid fuel consumption (kt)
CO2 emissions from solid fuel consumption (% of total)
Death rate, crude (per 1,000 people)
Fertility rate, total (births per woman)
Fixed telephone subscriptions
Fixed telephone subscriptions (per 100 people)
Hospital beds (per 1,000 people)
International migrant stock (% of population)
International migrant stock, total
Life expectancy at birth, female (years)
Life expectancy at birth, male (years)
Life expectancy at birth,

### Creating sub-dataset using Pivot table

In [104]:
world_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python Course/Large Data Sets/worlddata.csv')
world_data

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06
...,...,...,...,...,...,...
5656453,Zimbabwe,ZWE,Time required to register property (days),IC.PRP.DURS,2015,3.600000e+01
5656454,Zimbabwe,ZWE,Time required to start a business (days),IC.REG.DURS,2015,9.000000e+01
5656455,Zimbabwe,ZWE,Time to prepare and pay taxes (hours),IC.TAX.DURS,2015,2.420000e+02
5656456,Zimbabwe,ZWE,Time to resolve insolvency (years),IC.ISV.DURS,2015,3.300000e+00


In [105]:
# LifeExpCondition = world_data['IndicatorName']=="Life expectancy at birth, total (years)"
# FertilityRateCondition = world_data['IndicatorName']=="Fertility rate, total (births per woman)"
# BirthRateCondition = world_data['IndicatorName']=="Birth rate, crude (per 1,000 people)"
# wd_LifeExp_Fertility_BirthRate = world_data.loc[LifeExpCondition|FertilityRateCondition|BirthRateCondition]
# wd_LifeExp_Fertility_BirthRate

In [106]:
required_variables = ["Life expectancy at birth, total (years)", "Fertility rate, total (births per woman)", "Birth rate, crude (per 1,000 people)"]
wd_LifeExp_Fertility_BirthRate = world_data[world_data["IndicatorName"].isin(required_variables)]
wd_LifeExp_Fertility_BirthRate

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
6,Arab World,ARB,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,47.697888
14,Arab World,ARB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,6.924027
22,Arab World,ARB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,46.847059
84,Caribbean small states,CSS,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,40.017001
93,Caribbean small states,CSS,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,5.520103
...,...,...,...,...,...,...
5533015,Zambia,ZMB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2013,5.429000
5533191,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2013,59.237366
5533698,Zimbabwe,ZWE,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,2013,35.715000
5533801,Zimbabwe,ZWE,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2013,3.977000


In [107]:
wd_pivot=pd.pivot_table(wd_LifeExp_Fertility_BirthRate, values='Value', index=['CountryName','CountryCode', 'Year'], columns=['IndicatorName'])
wd_pivot

IndicatorName                 Birth rate, crude (per 1,000 people)  \
CountryName CountryCode Year                                         
Afghanistan AFG         1960                                51.276   
                        1961                                51.374   
                        1962                                51.464   
                        1963                                51.544   
                        1964                                51.614   
...                                                            ...   
Zimbabwe    ZWE         2009                                36.094   
                        2010                                36.267   
                        2011                                36.264   
                        2012                                36.077   
                        2013                                35.715   

IndicatorName                 Fertility rate, total (births per woman)  \
CountryName CountryCode Year                                             
Afghanistan AFG         1960                                     7.450   
                        1961                                     7.450   
                        1962                                     7.450   
                        1963                                     7.450   
                        1964                                     7.450   
...                                                                ...   
Zimbabwe    ZWE         2009                                     4.044   
                        2010                                     4.048   
                        2011                                     4.039   
                        2012                                     4.016   
                        2013                                     3.977   

IndicatorName                 Life expectancy at birth, total (years)  
CountryName CountryCode Year                                           
Afghanistan AFG         1960                                32.328512  
                        1961                                32.777439  
                        1962                                33.219902  
                        1963                                33.657878  
                        1964                                34.092878  
...                                                               ...  
Zimbabwe    ZWE         2009                                47.624659  
                        2010                                49.574659  
                        2011                                51.600366  
                        2012                                53.643073  
                        2013                                55.633000  

[12530 rows x 3 columns]

In [108]:
wd_merged_data = wd_pivot.reset_index()
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


In [109]:
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


In [110]:
wd_merged_data.reset_index(drop=True, inplace=True)
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


## Doing the same thing using "Merging" of datasets:



In [111]:
world_data[(world_data['CountryCode']=='USA')&(world_data['IndicatorName']=='Fertility rate, total (births per woman)')]

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
22268,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,3.6540
48745,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1961,3.6200
77124,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1962,3.4610
105741,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1963,3.3190
134779,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1964,3.1900
166709,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1965,2.9130
198077,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1966,2.7210
230795,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1967,2.5580
263350,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1968,2.4640
296335,United States,USA,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1969,2.4560


In [112]:
world_data.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


Obtain Life Expectancy Variable:

In [113]:
wd_LifeExp = world_data.loc[world_data['IndicatorName']=="Life expectancy at birth, total (years)"]
wd_LifeExp.rename(columns={"Value": "Life_Expectancy"},inplace=True)
wd_LifeExp.drop(columns=["IndicatorName", "IndicatorCode"],inplace=True)
wd_LifeExp

<ipython-input-113-f24ba5337271>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_LifeExp.rename(columns={"Value": "Life_Expectancy"},inplace=True)
<ipython-input-113-f24ba5337271>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wd_LifeExp.drop(columns=["IndicatorName", "IndicatorCode"],inplace=True)


,CountryName,CountryCode,Year,Life_Expectancy
22,Arab World,ARB,1960,46.847059
103,Caribbean small states,CSS,1960,62.271795
188,Central Europe and the Baltics,CEB,1960,67.823762
282,East Asia & Pacific (all income levels),EAS,1960,48.298317
400,East Asia & Pacific (developing only),EAP,1960,45.549658
...,...,...,...,...
5531448,Virgin Islands (U.S.),VIR,2013,79.624390
5531812,West Bank and Gaza,WBG,2013,73.203341
5532394,"Yemen, Rep.",YEM,2013,63.583512
5533191,Zambia,ZMB,2013,59.237366


Obtain Birth Rate Variable:

In [114]:
wd_BirthRate = world_data.loc[world_data['IndicatorName']=="Birth rate, crude (per 1,000 people)",["CountryCode","Year","Value"]]
wd_BirthRate.rename(columns={"Value": "Birth_Rate"},inplace=True)
wd_BirthRate

,CountryCode,Year,Birth_Rate
6,ARB,1960,47.697888
84,CSS,1960,40.017001
164,CEB,1960,19.117435
236,EAS,1960,26.362350
359,EAP,1960,26.635950
...,...,...,...
5531415,VIR,2013,10.700000
5531517,WBG,2013,30.394000
5532140,YEM,2013,32.947000
5532882,ZMB,2013,40.471000


In [115]:
wd_Fertility_Rate = world_data.loc[world_data['IndicatorName']=="Fertility rate, total (births per woman)",["CountryCode","Year","Value"]]
wd_Fertility_Rate.rename(columns={"Value": "Fertility_Rate"},inplace=True)
wd_Fertility_Rate

,CountryCode,Year,Fertility_Rate
14,ARB,1960,6.924027
93,CSS,1960,5.520103
182,CEB,1960,2.498618
259,EAS,1960,5.396794
372,EAP,1960,5.826618
...,...,...,...
5531418,VIR,2013,1.770000
5531600,WBG,2013,4.010000
5532252,YEM,2013,4.284000
5533015,ZMB,2013,5.429000


Merging:

In [116]:
wd_LifeExp_BirthRate = pd.merge(wd_LifeExp,wd_BirthRate, on=["CountryCode","Year"], how="outer")

In [117]:
wd_LifeExp_BirthRate_FertilitRate = pd.merge(wd_LifeExp_BirthRate, wd_Fertility_Rate, on = ["CountryCode","Year"], how = "outer")
wd_LifeExp_BirthRate_FertilitRate

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
0,Arab World,ARB,1960,46.847059,47.697888,6.924027
1,Caribbean small states,CSS,1960,62.271795,40.017001,5.520103
2,Central Europe and the Baltics,CEB,1960,67.823762,19.117435,2.498618
3,East Asia & Pacific (all income levels),EAS,1960,48.298317,26.362350,5.396794
4,East Asia & Pacific (developing only),EAP,1960,45.549658,26.635950,5.826618
...,...,...,...,...,...,...
12525,NaN,SRB,1981,NaN,NaN,2.100000
12526,NaN,SXM,1992,NaN,NaN,2.030000
12527,NaN,SXM,2001,NaN,NaN,1.950000
12528,NaN,DMA,2003,NaN,NaN,1.900000


Doing All merges in one step:

In [118]:
wd_merged = pd.merge(wd_LifeExp,wd_BirthRate, on=["CountryCode","Year"], how="outer").merge(wd_Fertility_Rate,on=["CountryCode","Year"], how="outer")
wd_merged

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
0,Arab World,ARB,1960,46.847059,47.697888,6.924027
1,Caribbean small states,CSS,1960,62.271795,40.017001,5.520103
2,Central Europe and the Baltics,CEB,1960,67.823762,19.117435,2.498618
3,East Asia & Pacific (all income levels),EAS,1960,48.298317,26.362350,5.396794
4,East Asia & Pacific (developing only),EAP,1960,45.549658,26.635950,5.826618
...,...,...,...,...,...,...
12525,NaN,SRB,1981,NaN,NaN,2.100000
12526,NaN,SXM,1992,NaN,NaN,2.030000
12527,NaN,SXM,2001,NaN,NaN,1.950000
12528,NaN,DMA,2003,NaN,NaN,1.900000


In [119]:
wd_merged.shape

(12530, 6)

In [120]:
wd_merged.dropna(how="any")

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
0,Arab World,ARB,1960,46.847059,47.697888,6.924027
1,Caribbean small states,CSS,1960,62.271795,40.017001,5.520103
2,Central Europe and the Baltics,CEB,1960,67.823762,19.117435,2.498618
3,East Asia & Pacific (all income levels),EAS,1960,48.298317,26.362350,5.396794
4,East Asia & Pacific (developing only),EAP,1960,45.549658,26.635950,5.826618
...,...,...,...,...,...,...
12299,Virgin Islands (U.S.),VIR,2013,79.624390,10.700000,1.770000
12300,West Bank and Gaza,WBG,2013,73.203341,30.394000,4.010000
12301,"Yemen, Rep.",YEM,2013,63.583512,32.947000,4.284000
12302,Zambia,ZMB,2013,59.237366,40.471000,5.429000


In [121]:
wd_merged.CountryName.unique()

array(['Arab World', 'Caribbean small states',
       'Central Europe and the Baltics',
       'East Asia & Pacific (all income levels)',
       'East Asia & Pacific (developing only)', 'Euro area',
       'Europe & Central Asia (all income levels)',
       'Europe & Central Asia (developing only)', 'European Union',
       'Fragile and conflict affected situations',
       'Heavily indebted poor countries (HIPC)', 'High income',
       'High income: nonOECD', 'High income: OECD',
       'Latin America & Caribbean (all income levels)',
       'Latin America & Caribbean (developing only)',
       'Least developed countries: UN classification',
       'Low & middle income', 'Low income', 'Lower middle income',
       'Middle East & North Africa (all income levels)',
       'Middle East & North Africa (developing only)', 'Middle income',
       'North America', 'OECD members', 'Other small states',
       'Pacific island small states', 'Small states', 'South Asia',
       'Sub-Saharan Afr

In [122]:
wd_merged[wd_merged['CountryName']=="United States"]

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
212,United States,USA,1960,69.770732,23.7,3.6540
435,United States,USA,1961,70.270732,23.3,3.6200
658,United States,USA,1962,70.119512,22.4,3.4610
880,United States,USA,1963,69.917073,21.7,3.3190
1102,United States,USA,1964,70.165854,21.1,3.1900
1325,United States,USA,1965,70.214634,19.4,2.9130
1548,United States,USA,1966,70.212195,18.4,2.7210
1771,United States,USA,1967,70.560976,17.8,2.5580
1994,United States,USA,1968,69.951220,17.6,2.4640
2217,United States,USA,1969,70.507317,17.9,2.4560


In [123]:
wd_merged[(wd_merged['CountryName']=="United States")& (wd_merged['Year']==1960.0)]

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
212,United States,USA,1960,69.770732,23.7,3.654


In [124]:
wd_merged[(wd_merged['CountryName']=="India")& (wd_merged['Year']==1960.0)]

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
111,India,IND,1960,41.171951,42.107,5.906


In [125]:
wd_merged[(wd_merged['CountryName']=="China")]

,CountryName,CountryCode,Year,Life_Expectancy,Birth_Rate,Fertility_Rate
69,China,CHN,1960,43.465805,20.86,5.758
291,China,CHN,1961,43.780756,18.02,5.887
514,China,CHN,1962,44.712659,37.01,6.008
737,China,CHN,1963,46.331000,43.37,6.103
959,China,CHN,1964,48.580878,39.14,6.157
1182,China,CHN,1965,51.291439,37.88,6.161
1404,China,CHN,1966,54.204122,35.05,6.110
1627,China,CHN,1967,57.012390,33.96,6.011
1850,China,CHN,1968,59.468854,35.59,5.871
2073,China,CHN,1969,61.451171,34.11,5.692


### Another method using Pivot table

In [126]:
world_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python Course/Large Data Sets/worlddata.csv')
world_data

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06
...,...,...,...,...,...,...
5656453,Zimbabwe,ZWE,Time required to register property (days),IC.PRP.DURS,2015,3.600000e+01
5656454,Zimbabwe,ZWE,Time required to start a business (days),IC.REG.DURS,2015,9.000000e+01
5656455,Zimbabwe,ZWE,Time to prepare and pay taxes (hours),IC.TAX.DURS,2015,2.420000e+02
5656456,Zimbabwe,ZWE,Time to resolve insolvency (years),IC.ISV.DURS,2015,3.300000e+00


In [127]:
# LifeExpCondition = world_data['IndicatorName']=="Life expectancy at birth, total (years)"
# FertilityRateCondition = world_data['IndicatorName']=="Fertility rate, total (births per woman)"
# BirthRateCondition = world_data['IndicatorName']=="Birth rate, crude (per 1,000 people)"
# wd_LifeExp_Fertility_BirthRate = world_data.loc[LifeExpCondition|FertilityRateCondition|BirthRateCondition]
# wd_LifeExp_Fertility_BirthRate

In [128]:
required_variables = ["Life expectancy at birth, total (years)", "Fertility rate, total (births per woman)", "Birth rate, crude (per 1,000 people)"]
wd_LifeExp_Fertility_BirthRate = world_data[world_data["IndicatorName"].isin(required_variables)]
wd_LifeExp_Fertility_BirthRate

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
6,Arab World,ARB,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,47.697888
14,Arab World,ARB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,6.924027
22,Arab World,ARB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,1960,46.847059
84,Caribbean small states,CSS,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,40.017001
93,Caribbean small states,CSS,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,1960,5.520103
...,...,...,...,...,...,...
5533015,Zambia,ZMB,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2013,5.429000
5533191,Zambia,ZMB,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,2013,59.237366
5533698,Zimbabwe,ZWE,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,2013,35.715000
5533801,Zimbabwe,ZWE,"Fertility rate, total (births per woman)",SP.DYN.TFRT.IN,2013,3.977000


In [129]:
wd_pivot=pd.pivot_table(wd_LifeExp_Fertility_BirthRate, values='Value', index=['CountryName','CountryCode', 'Year'], columns=['IndicatorName'])
wd_pivot

IndicatorName                 Birth rate, crude (per 1,000 people)  \
CountryName CountryCode Year                                         
Afghanistan AFG         1960                                51.276   
                        1961                                51.374   
                        1962                                51.464   
                        1963                                51.544   
                        1964                                51.614   
...                                                            ...   
Zimbabwe    ZWE         2009                                36.094   
                        2010                                36.267   
                        2011                                36.264   
                        2012                                36.077   
                        2013                                35.715   

IndicatorName                 Fertility rate, total (births per woman)  \
CountryName CountryCode Year                                             
Afghanistan AFG         1960                                     7.450   
                        1961                                     7.450   
                        1962                                     7.450   
                        1963                                     7.450   
                        1964                                     7.450   
...                                                                ...   
Zimbabwe    ZWE         2009                                     4.044   
                        2010                                     4.048   
                        2011                                     4.039   
                        2012                                     4.016   
                        2013                                     3.977   

IndicatorName                 Life expectancy at birth, total (years)  
CountryName CountryCode Year                                           
Afghanistan AFG         1960                                32.328512  
                        1961                                32.777439  
                        1962                                33.219902  
                        1963                                33.657878  
                        1964                                34.092878  
...                                                               ...  
Zimbabwe    ZWE         2009                                47.624659  
                        2010                                49.574659  
                        2011                                51.600366  
                        2012                                53.643073  
                        2013                                55.633000  

[12530 rows x 3 columns]

In [130]:
wd_merged_data = wd_pivot.reset_index()
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


In [131]:
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


In [132]:
wd_merged_data.reset_index(drop=True, inplace=True)
wd_merged_data

IndicatorName,CountryName,CountryCode,Year,"Birth rate, crude (per 1,000 people)","Fertility rate, total (births per woman)","Life expectancy at birth, total (years)"
0,Afghanistan,AFG,1960,51.276,7.450,32.328512
1,Afghanistan,AFG,1961,51.374,7.450,32.777439
2,Afghanistan,AFG,1962,51.464,7.450,33.219902
3,Afghanistan,AFG,1963,51.544,7.450,33.657878
4,Afghanistan,AFG,1964,51.614,7.450,34.092878
...,...,...,...,...,...,...
12525,Zimbabwe,ZWE,2009,36.094,4.044,47.624659
12526,Zimbabwe,ZWE,2010,36.267,4.048,49.574659
12527,Zimbabwe,ZWE,2011,36.264,4.039,51.600366
12528,Zimbabwe,ZWE,2012,36.077,4.016,53.643073


***
<a id='section_id7'></a>
### <span style="background-color:#B0E0E6">Additional References</span>

 - The [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html) is a good resource that provides a
 long list of [Pandas Tutorials](https://pandas.pydata.org/pandas-docs/stable/getting_started/tutorials.html)
 - Chapter 5 of [Python for Data Analysis 2<sup>nd</sup> edition](https://pitt.primo.exlibrisgroup.com/permalink/01PITT_INST/1sjtb5p/alma9998559287406236) by Wes McKinney
 - Try this [collection of Pandas exercises](https://github.com/guipsamora/pandas_exercises) recommended for new Pandas users.
 - Go through these examples on [data munging](http://wavedatalab.github.io/datawithpython/munge.html) and [data aggregation](http://wavedatalab.github.io/datawithpython/aggregate.html)
 - A very useful series of [lessons on effective Pandas](https://github.com/TomAugspurger/effective-pandas)
 - This set of [100 Pandas puzzles](https://github.com/ajcr/100-pandas-puzzles) also looks interesting; try them out.
 ***

<font color="#610B21"><center>\~\~ End of Session-3. We'll learn about exploratory analysis and data visualization in Session-4\~\~ </center>
<font color="#0B610B"><center>\*\* **Good luck with your practice!**\*\* </center>
***

<div style="text-align: right"> <a href='#section_id0'>Back to top of Notebook</a></div>